In [ ]:
# general imports
import torch as th
import torch.nn.functional as F
from torch_geometric.loader import DataLoader as PygDataLoader
from torch.utils.data import SequentialSampler

In [ ]:
import torch
from torch.nn import CrossEntropyLoss
from torch_geometric.data import DataLoader
from dataloader import NCaltech101Best
from model import GraphRes, GraphResModified, GraphResSimple, GraphTrans, GraphTransFinal
from tqdm import tqdm

In [ ]:
# params
batch_size= 16
lr=         10e-3 # decreases by 10 after each 20 epochs
loss=       th.nn.CrossEntropyLoss
batchsize=  16
K=          10 # data subsampling
nclasses=   100

In [ ]:
%env CUDA_LAUNCH_BLOCKING=1

In [ ]:
# Set device to use for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("using device:", device)

# Initialize dataset and data loader
dataset = NCaltech101Best('./data/storage/', mode='train', num_samples=10000)
# sampler = SequentialSampler(dataset)
# loader = DataLoader(dataset, batch_size=16, sampler=sampler)
loader = PygDataLoader(dataset, batch_size=32, shuffle=True)
print("example datapoint:", dataset.get(0))


# Initialize model
model_input_shape = th.tensor((240, 180) + (3, ), device=device)
print("INPUT SHAPE:", model_input_shape)
model = GraphTransFinal('ncaltech101', model_input_shape, 101, dropout_trans=True, heads=1).to(device)

# Define loss function and optimizer
criterion = CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# for testing
test_dataset = NCaltech101Best('./data/storage/', mode='test', num_samples=10000)
test_loader = PygDataLoader(test_dataset, batch_size=32, shuffle=True)
def run_model_test(model, loader):
    model.eval()
    correct = 0
    for data in tqdm(loader):
        data = data.to(device)
        with torch.no_grad():
            out = model(data)
            pred = out.max(dim=1)[1]
            correct += pred.eq(data.y).sum().item()
    return correct / len(loader.dataset)


acc = []
test_acc = []
# Define training loop
def train():
    model.train()
    losses = []
    correct = 0
    i = 0
    print("234 iterations in total:")
    progbar = tqdm(loader)
    for data in progbar:

        # inference
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)

        # weight updates
        y = F.one_hot(data.y, num_classes=101).type(torch.cuda.FloatTensor)
        loss = criterion(output, y)
        losses.append(loss)
        loss.backward()
        optimizer.step()

        # precision logging
        pred = output.max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
        i += 1
        # if i % 50 == 0:
        #     print(correct, i*16)
        #     print(correct / (i*16))
        # if i % 20 == 0 and i != 0:
        #     print((i, round(correct / (i*16), 2)), end='->')
        # if i % 100 == 0 and i != 0:   
        #     print("\n", end='')
        precision = round(correct / (i*32), 2)
        progbar.set_description("precision so far is {}".format(precision))
    
    # test, to check for overfitting
    tacc = run_model_test(model, test_loader)
    test_acc.append(tacc)
    
    acc.append(correct / (len(loader) * 32))
    print("train accuracy: {}, test accuracy: {}".format(acc[len(acc) - 1], tacc))
    return losses

# Run training loop for 10 epochs
all_losses = []
best_tacc = 0.0
for epoch in range(100):
    print("computing epoch", epoch)
    losses = train()
    if test_acc[-1] > best_tacc:
        torch.save(model, "trained_graphtransfinal_best.model")
        best_tacc = test_acc[-1]
    all_losses.append(losses)

In [ ]:
print(all_losses[0][0].cpu().detach().numpy())
def clean_losses(losses):
    clean = []
    for loss in losses:
        ll = [float(l.cpu().detach().numpy()) for l in loss]
        clean.append(ll)
    return clean
print(clean_losses(all_losses))
print(test_acc)
print(max(test_acc))

In [ ]:
# test
dataset = NCaltech101Best('./data/storage/', mode='test')
loader = PygDataLoader(dataset, batch_size=16, shuffle=True)
def test(model, loader):
    model.eval()

    correct = 0
    for data in tqdm(loader):
        data = data.to(device)
        with torch.no_grad():
            out = model(data)
            pred = out.max(dim=1)[1]
            correct += pred.eq(data.y).sum().item()
    return correct / len(loader.dataset)

print(test(model,loader))

In [ ]:
torch.save(model, "trained_graphtrans.model")

In [ ]:
len(all_losses)

In [ ]:
# all_losses2 = [l.cpu().detach().numpy() for l in [**all_losses]]

In [ ]:
# what to do with the losses...

list_losses = []
for losses in all_losses:
    list_losses += losses

list_losses2 = [l.cpu().detach().numpy() for l in list_losses]

import matplotlib as mpl
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot(range(len(list_losses2)), list_losses2)
fig.show()